<a href="https://colab.research.google.com/github/olbp07/olbp07/blob/main/Simulador%20de%20Renda%20FIxa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install PyQt5

import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import Workbook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from PyQt5.QtWidgets import (QApplication, QWidget, QLabel, QLineEdit, QPushButton, QVBoxLayout, QHBoxLayout,
                             QComboBox, QCheckBox, QRadioButton, QSlider, QGridLayout, QMessageBox)
from PyQt5.QtCore import Qt
from functools import lru_cache
from openpyxl.utils.dataframe import dataframe_to_rows

@lru_cache(maxsize=None)
def calcular_imposto_renda(montante, aporte_mensal, mes, produto):
    """
    Calcula o imposto de renda devido com base no montante total,
    aporte mensal, número de meses que o investimento foi mantido e o produto.
    Para LCI e LCA, o imposto é zero.
    """
    rendimento = montante - (aporte_mensal * mes)

    # Verifica se o produto é LCI ou LCA
    if produto in ("LCI", "LCA"):
        aliquota = 0  # Isento de imposto de renda
    elif mes <= 6:
        aliquota = 0.225  # 22,5%
    elif mes <= 12:
        aliquota = 0.20  # 20%
    elif mes <= 24:
        aliquota = 0.175  # 17,5%
    else:
        aliquota = 0.15  # 15%

    imposto = rendimento * aliquota
    return imposto

def calcular_taxa_administrativa(montante, taxa_administrativa, tipo_taxa):
    """Calcula a taxa administrativa mensal ou anual."""
    if tipo_taxa == "mensal":
        taxa = montante * taxa_administrativa
    elif tipo_taxa == "anual":
        taxa = (montante * taxa_administrativa) / 12  # Divide a taxa anual por 12 para obter a taxa mensal
    else:
        raise ValueError("Tipo de taxa inválido. Use 'mensal' ou 'anual'.")
    return taxa

def calcular_montante_com_ir(aporte_mensal, taxa_juros, num_meses, produto, taxa_administrativa=0, tipo_taxa=None,
                              taxa_inflacao=0):
    """Simula o investimento com NumPy."""

    meses = np.arange(1, num_meses + 1)
    montantes = np.zeros(num_meses)
    montantes[0] = aporte_mensal * (1 + taxa_juros)
    for mes in range(1, num_meses):
        montantes[mes] = (montantes[mes - 1] + aporte_mensal) * (1 + taxa_juros)

    # Calcula o imposto de renda com NumPy
    aliquotas = np.where(meses <= 6, 0.225,
                         np.where(meses <= 12, 0.20,
                                  np.where(meses <= 24, 0.175, 0.15)))
    if produto in ("LCI", "LCA"):
        aliquotas = 0
    impostos = (montantes - aporte_mensal * meses) * aliquotas

    # Calcula a taxa administrativa com NumPy
    if taxa_administrativa:
        if tipo_taxa == "mensal":
            taxas_administrativas = montantes * taxa_administrativa
        else:
            taxas_administrativas = (montantes * taxa_administrativa) / 12
        montantes -= taxas_administrativas

    montantes -= impostos
    montantes *= (1 + taxa_inflacao)

    return montantes[-1], impostos, impostos / montantes * 100, montantes, taxas_administrativas

def criar_tabela_mensal(resultados, cenario, produto):
    """
    Organiza os resultados do cálculo do imposto de renda em um
    DataFrame do pandas, criando uma tabela com as colunas "Mês",
    "Montante Acumulado", "Imposto de Renda", "% IR" e "Taxa Administrativa".
    """
    df_mensal = pd.DataFrame(columns=['Mês', 'Montante Acumulado', 'Imposto de Renda', '% IR', 'Taxa Administrativa'])
    num_meses = len(resultados[cenario][produto]['Montantes Acumulados'])
    for mes in range(1, num_meses + 1):
        montante_acumulado = resultados[cenario][produto]['Montantes Acumulados'][mes - 1]
        imposto = resultados[cenario][produto]['Impostos'][mes - 1]
        porcentagem_ir = resultados[cenario][produto]['Porcentagens IR'][mes - 1]
        # Verifica se a lista de taxas administrativas existe e se o índice é válido
        if 'Taxas Administrativas' in resultados[cenario][produto] and mes - 1 < len(
                resultados[cenario][produto]['Taxas Administrativas']):
            taxa_administrativa = resultados[cenario][produto]['Taxas Administrativas'][mes - 1]
        else:
            taxa_administrativa = 0
        df_mensal = pd.concat([df_mensal, pd.DataFrame({
            'Mês': [mes],
            'Montante Acumulado': [f"R$ {montante_acumulado:.2f}"],
            'Imposto de Renda': [f"R$ {imposto:.2f}"],
            '% IR': [f"{porcentagem_ir:.2f}%"],
            'Taxa Administrativa': [f"R$ {taxa_administrativa:.2f}"]
        })], ignore_index=True)

    return df_mensal

def obter_taxa_selic():
    """Obtém a taxa Selic atual do site do Banco Central do Brasil."""
    url = "https://www.bcb.gov.br/controleinflacao/taxaselic"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Lança uma exceção para erros HTTP (ex: 404)
        soup = BeautifulSoup(response.content, "html.parser")
        # Encontra a taxa Selic na página (ajuste o seletor CSS se necessário)
        taxa_selic_elemento = soup.select_one(
            ".conteudo-historico-taxas .col-md-12:nth-child(2) tr:nth-child(2) td:nth-child(2)")  # noqa: E501
        if taxa_selic_elemento:
            taxa_selic_str = taxa_selic_elemento.text.strip().replace(",", ".")
            taxa_selic = float(taxa_selic_str) / 100  # Converte para decimal
            return taxa_selic
        else:
            print("Erro: Elemento da taxa Selic não encontrado na página.")
            return None
    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar o site do Banco Central: {e}")
        return None
    except ValueError as e:
        print(f"Erro ao converter a taxa Selic para número: {e}")
        return None

def calcular_investimentos(aporte_mensal, num_meses, taxa_administrativa, tipo_taxa, taxa_selic):
    """Calcula os investimentos para cada cenário e produto."""
    cenarios = {
        "Conservador": {
            "Tesouro Selic": taxa_selic + 0.0005,
            "CDB pré-fixado": 0.0008,
            "CDB pós-fixado": 0.0008,
            "LCI": taxa_selic + 0.0007,
            "LCA": taxa_selic + 0.0007,
            "Fundos Imobiliários": 0.0004,
        },
        "Otimista": {
            "Tesouro Selic": taxa_selic + 0.0008,
            "CDB pré-fixado": 0.0011,
            "CDB pós-fixado": 0.0011,
            "LCI": taxa_selic + 0.0010,
            "LCA": taxa_selic + 0.0010,
            "Fundos Imobiliários": 0.0008,
        },
    }

    resultados = {
        "Conservador": {},
        "Otimista": {}
    }

    for cenario, taxas in cenarios.items():
        for produto, taxa in taxas.items():
            # Passa os parâmetros da taxa administrativa para a função calcular_montante_com_ir
            montante, impostos, porcentagens_ir, montantes_acumulados, taxas_administrativas = calcular_montante_com_ir(
                aporte_mensal, taxa, num_meses, produto, taxa_administrativa, tipo_taxa
            )
            resultados[cenario][produto] = {
                "Montante": montante,
                "Impostos": impostos,
                "Porcentagens IR": porcentagens_ir,
                "Montantes Acumulados": montantes_acumulados,
                "Taxas Administrativas": taxas_administrativas
            }
    return resultados

def exibir_resultados(self, resultados, num_meses):
    """Exibe os resultados da simulação, incluindo gráficos."""
    self.figure.clear()

    # Cria um dicionário para armazenar os DataFrames dos resultados
    dfs = {}

    for cenario in resultados:
        print(f"\nCenário {cenario}:")
        for produto in resultados[cenario]:
            print(f"\n{produto}:")
            if produto in ("CDB pré-fixado", "CDB pós-fixado"):
                print(
                    "Incide Imposto de Renda sobre os rendimentos, com alíquotas regressivas de acordo com o tempo de investimento:\n"
                    "Até 180 dias: 22,5%\n"
                    "De 181 a 360 dias: 20%\n"
                    "De 361 a 720 dias: 17,5%\n"
                    "Acima de 720 dias: 15%\n"
                )
            df_mensal = criar_tabela_mensal(resultados, cenario, produto)

            # Armazena o DataFrame no dicionário
            dfs[f'{cenario} - {produto}'] = df_mensal

            display(df_mensal.style.hide(axis='index'))

    print("\nOs valores em negrito indicam o valor final do investimento após",
          f"{num_meses} meses, já deduzido o Imposto de Renda e a Taxa Administrativa (se houver).")
    print("As tabelas mostram o valor do IR devido e a porcentagem do IR sobre",
          "o montante total a cada mês.")
    print("É importante lembrar que o IR incide apenas sobre os rendimentos, e",
          "não sobre o valor total investido.")

    # Exporta para Excel
    exportar = input("Deseja exportar os resultados para Excel? (s/n): ")
    if exportar.lower() == "s":
        wb = Workbook()
        for cenario in resultados:
            ws = wb.create_sheet(cenario)
            for i, produto in enumerate(resultados[cenario]):
                ws.cell(row=1, column=i * 4 + 1).value = produto
                df_mensal = criar_tabela_mensal(resultados, cenario, produto)
                for row in dataframe_to_rows(df_mensal, index=False, header=True):
                    ws.append(row)
        wb.save("resultados_simulcao.xlsx")
        print("Resultados exportados para resultados_simulcao.xlsx")

    # Exibe os gráficos na interface PyQt
    self.plotar_graficos(dfs)

def plotar_graficos(self, dfs):
    """Plota os gráficos na interface PyQt."""

    # Define o layout para os gráficos
    layout_graficos = QGridLayout()

    # Cria um gráfico para cada DataFrame
    for i, (titulo, df) in enumerate(dfs.items()):
        grafico = plt.figure(figsize=(5, 3))
        canvas = FigureCanvas(grafico)
        ax = grafico.add_subplot(111)
        ax.plot(df['Mês'], df['Montante Acumulado'], label=titulo)
        ax.set_xlabel("Mês")
        ax.set_ylabel("Montante Acumulado")
        ax.legend()
        ax.grid(True)

        # Adiciona o canvas ao layout
        layout_graficos.addWidget(canvas, i // 2, i % 2)

    # Remove o layout antigo (se existir)
    if self.layout() is not None:
        QWidget().setLayout(self.layout())

    # Define o novo layout
    self.setLayout(layout_graficos)

class SimuladorInvestimentos(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Simulador de Investimentos")
        self.setGeometry(100, 100, 800, 600)

        # Layout principal
        layout_principal = QVBoxLayout()

        # Layout para os inputs
        layout_inputs = QGridLayout()

        # Aporte mensal
        self.label_aporte = QLabel("Aporte Mensal:")
        self.edit_aporte = QLineEdit()
        layout_inputs.addWidget(self.label_aporte, 0, 0)
        layout_inputs.addWidget(self.edit_aporte, 0, 1)

        # Número de meses
        self.label_meses = QLabel("Número de Meses:")
        self.combo_meses = QComboBox()
        self.combo_meses.addItems(["12", "24", "36", "48", "60"])
        layout_inputs.addWidget(self.label_meses, 1, 0)
        layout_inputs.addWidget(self.combo_meses, 1, 1)

        # Taxa administrativa
        self.check_taxa = QCheckBox("Incluir Taxa Administrativa")
        self.label_taxa = QLabel("Taxa Administrativa:")
        self.edit_taxa = QLineEdit()
        self.edit_taxa.setEnabled(False)
        self.combo_tipo_taxa = QComboBox()
        self.combo_tipo_taxa.addItems(["mensal", "anual"])
        self.combo_tipo_taxa.setEnabled(False)
        layout_inputs.addWidget(self.check_taxa, 2, 0, 1, 2)
        layout_inputs.addWidget(self.label_taxa, 3, 0)
        layout_inputs.addWidget(self.edit_taxa, 3, 1)
        layout_inputs.addWidget(self.combo_tipo_taxa, 4, 1)

        # Taxa Selic
        self.label_selic = QLabel("Taxa Selic:")
        self.radio_selic_manual = QRadioButton("Digitar")
        self.radio_selic_automatica = QRadioButton("Automática")
        self.radio_selic_automatica.setChecked(True)
        self.edit_selic = QLineEdit()
        self.edit_selic.setEnabled(False)
        layout_inputs.addWidget(self.label_selic, 5, 0)
        layout_inputs.addWidget(self.radio_selic_manual, 6, 0)
        layout_inputs.addWidget(self.radio_selic_automatica, 7, 0)
        layout_inputs.addWidget(self.edit_selic, 6, 1, 1, 2)

        # Botão para simular
        self.botao_simular = QPushButton("Simular")
        layout_inputs.addWidget(self.botao_simular, 8, 0, 1, 2)

        layout_principal.addLayout(layout_inputs)

        # Área para exibir os resultados
        self.label_resultados = QLabel("Resultados:")
        layout_principal.addWidget(self.label_resultados)

        # Cria a área para o gráfico
        self.figure = plt.figure(figsize=(5, 3))
        self.canvas = FigureCanvas(self.figure)
        layout_principal.addWidget(self.canvas)

        self.setLayout(layout_principal)

        # Conecta os eventos
        self.botao_simular.clicked.connect(self.simular_investimentos)
        self.check_taxa.stateChanged.connect(self.habilitar_taxa)
        self.radio_selic_manual.toggled.connect(self.habilitar_selic)

    def habilitar_taxa(self):
        if self.check_taxa.isChecked():
            self.edit_taxa.setEnabled(True)
            self.combo_tipo_taxa.setEnabled(True)
        else:
            self.edit_taxa.setEnabled(False)
            self.combo_tipo_taxa.setEnabled(False)

    def habilitar_selic(self):
        if self.radio_selic_manual.isChecked():
            self.edit_selic.setEnabled(True)
        else:
            self.edit_selic.setEnabled(False)

    def simular_investimentos(self):
        try:
            aporte_mensal = float(self.edit_aporte.text())
            num_meses = int(self.combo_meses.currentText())
            taxa_administrativa = 0
            tipo_taxa = None
            if self.check_taxa.isChecked():
                taxa_administrativa = float(self.edit_taxa.text())
                tipo_taxa = self.combo_tipo_taxa.currentText()
            if self.radio_selic_manual.isChecked():
                taxa_selic = float(self.edit_selic.text()) / 100  # Divide por 100 para obter o valor decimal
            else:
                taxa_selic = self.obter_taxa_selic()
                if taxa_selic is None:
                    QMessageBox.warning(self, "Erro", "Não foi possível obter a taxa Selic automaticamente.")
                    return

            resultados = self.calcular_investimentos(aporte_mensal, num_meses, taxa_administrativa, tipo_taxa,
                                                        taxa_selic)
            self.exibir_resultados(resultados, num_meses)

        except ValueError:
            QMessageBox.warning(self, "Erro", "Por favor, insira valores válidos.")

    def obter_taxa_selic(self):
        """Obtém a taxa Selic atual do site do Banco Central do Brasil."""
        url = "https://www.bcb.gov.br/controleinflacao/taxaselic"
        try:
            response = requests.get(url)
            response.raise_for_status()  # Lança uma exceção para erros HTTP (ex: 404)
            soup = BeautifulSoup(response.content, "html.parser")
            # Encontra a taxa Selic na página (ajuste o seletor CSS se necessário)
            taxa_selic_elemento = soup.select_one(
                ".conteudo-historico-taxas .col-md-12:nth-child(2) tr:nth-child(2) td:nth-child(2)")  # noqa: E501
            if taxa_selic_elemento:
                taxa_selic_str = taxa_selic_elemento.text.strip().replace(",", ".")
                taxa_selic = float(taxa_selic_str) / 100  # Converte para decimal
                return taxa_selic
            else:
                print("Erro: Elemento da taxa Selic não encontrado na página.")
                return None
        except requests.exceptions.RequestException as e:
            print(f"Erro ao acessar o site do Banco Central: {e}")
            return None
        except ValueError as e:
            print(f"Erro ao converter a taxa Selic para número: {e}")
            return None

    def calcular_investimentos(self, aporte_mensal, num_meses, taxa_administrativa, tipo_taxa, taxa_selic):
        """Calcula os investimentos para cada cenário e produto."""
        cenarios = {
            "Conservador": {
                "Tesouro Selic": taxa_selic + 0.0005,
                "CDB pré-fixado": 0.0008,
                "CDB pós-fixado": 0.0008,
                "LCI": taxa_selic + 0.0007,
                "LCA": taxa_selic + 0.0007,
                "Fundos Imobiliários": 0.0004,
            },
            "Otimista": {
                "Tesouro Selic": taxa_selic + 0.0008,
                "CDB pré-fixado": 0.0011,
                "CDB pós-fixado": 0.0011,
                "LCI": taxa_selic + 0.0010,
                "LCA": taxa_selic + 0.0010,
                "Fundos Imobiliários": 0.0008,
            },
        }

        resultados = {
            "Conservador": {},
            "Otimista": {}
        }

        for cenario, taxas in cenarios.items():
            for produto, taxa in taxas.items():
                # Passa os parâmetros da taxa administrativa para a função calcular_montante_com_ir
                montante, impostos, porcentagens_ir, montantes_acumulados, taxas_administrativas = calcular_montante_com_ir(
                    aporte_mensal, taxa, num_meses, produto, taxa_administrativa, tipo_taxa
                )
                resultados[cenario][produto] = {
                    "Montante": montante,
                    "Impostos": impostos,
                    "Porcentagens IR": porcentagens_ir,
                    "Montantes Acumulados": montantes_acumulados,
                    "Taxas Administrativas": taxas_administrativas
                }
        return resultados

    def exibir_resultados(self, resultados, num_meses):
        # ... (código da função exibir_resultados)

        if __name__ == '__main__':
           app = QApplication(sys.argv)
           simulador = SimuladorInvestimentos()
           simulador.show()
           sys.exit(app.exec_())